In [ ]:
import airpy as ap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

bookings = pd.DataFrame(pd.read_csv('/Users/kenji_hagiwara/Downloads/bookings.csv', index_col='calendar_date', parse_dates=True, infer_datetime_format=True,)).loc['2018-08-01':'2018-08-31']#.reset_index()
costs = pd.DataFrame(pd.read_csv('/Users/kenji_hagiwara/Downloads/costs.csv', index_col='calendar_date', parse_dates=True, infer_datetime_format=True,)).loc['2018-08-01':'2018-08-31']#.reset_index()
revenue = pd.DataFrame(pd.read_csv('/Users/kenji_hagiwara/Downloads/revenue.csv', index_col='calendar_date', parse_dates=True, infer_datetime_format=True,)).loc['2018-08-01':'2018-08-31']#.reset_index()


result = pd.merge(costs, bookings, on=['tracking_key','calendar_date'])
result = pd.merge(result, revenue, on=['tracking_key','calendar_date'])

#Regressions
result.plot(x='bookings', y='cost', kind='scatter', alpha=0.1)
plt.show()

result.plot(x='clicks', y='revenue', kind='scatter', alpha=0.1)
plt.show()

result.plot(x='bookings', y='clicks', kind='scatter', alpha=0.1)
plt.show()

result.plot(x='revenue', y='clicks', kind='scatter', alpha=0.1)
plt.show()

In [ ]:
#slice by campaign strategy
sns.lmplot(x="bookings", y="cost", col="campaign_strategy", hue="campaign_strategy", data=result, col_wrap=2)
plt.show()




In [ ]:
#slice by platform & campaign strategy
sns.lmplot(x="bookings", y="cost", col="campaign_strategy", hue="platform", 
          data=result, col_wrap=2,
          line_kws={"alpha":0.1},
          scatter_kws={"alpha": 0.1})

plt.show()



In [ ]:
# get all columns from dataframe that are measure columns
#num_cols = result.select_dtypes('float64').columns.values

# get all columns from dataframe that are dimensional columns 
#non_num_cols = result.select_dtypes('object').columns.values[1:]

# manually define slices that we care about
num_cols = ['cost','clicks','revenue']
non_num_cols = ['platform', 'campaign_strategy', 'channel', 'dim_origin_region']


for x in num_cols: #dynamically slices x and y axis by each measure (iterates through every combination)
   for y in num_cols: 
       for col in non_num_cols: 
           #for hue in non_num_cols:   #dynamically slices with each dimension
               print (x,y,col,hue) #prints title of graph set
               if (x != y) & (col != hue): #prevents redundant slice combinations like impressions vs impressions. #
                   sns.lmplot(x=x, y=y, col=col, hue=hue, 
                              data=result, col_wrap=3,
                              line_kws={"alpha":0.1},
                              scatter_kws={"alpha": 0.1})
                   plt.show()


In [ ]:
result_log = result.copy()
result_log[num_cols] = np.log(result_log[num_cols]) #applying np.log to numerical columns in result_log (this scales everything down, so we can see the relationships more clearly)

for x in num_cols:
   for y in num_cols:
       for col in non_num_cols:
           for hue in non_num_cols:
               print (x,y,col,hue)
               # dont do redundant x vs x plots or col=hue plots
               if (x != y) & (col != hue):
                   sns.lmplot(x=x, y=y, col=col, hue=hue, 
                              data=result_log, col_wrap=2,
                              line_kws={"alpha":0.5},
                              scatter_kws={"alpha": 0.9, 's': 10})
                   plt.show()
                    
                    #Look into these further: 
                    #seeing clustering in low intent cost/clicks in social
                    #clustering in app brand, app installs, cost/clicks. 
                    
                    #corelation between Google and Revenue
                    #Correlation between origon_region Australia and revenue 


In [ ]:
australia_google = result[(result['dim_origin_region'] == 'Australia') & (result['platform'] == 'google')]



num_cols = ['clicks','revenue']
non_num_cols = ['campaign_strategy', 'channel']


for x in num_cols: #dynamically slices x and y axis by each measure (iterates through every combination)
   for y in num_cols: 
       for col in non_num_cols: 
               if (x != y) : #prevents redundant slice combinations like impressions vs impressions. #
                   sns.lmplot(x=x, y=y, col=col,  
                              data=australia_google, col_wrap=2,
                              line_kws={"alpha":0.1},
                              scatter_kws={"alpha": 0.1})
                   plt.show()

In [ ]:
australia_google_SEMNB = result[(result['dim_origin_region'] == 'Australia') & (result['platform'] == 'google') & (result['channel'] == 'SEM Non-brand')]

sns.lmplot(x='clicks', y='revenue',
        data=australia_google_SEMNB,
        line_kws={"alpha":0.5},
        scatter_kws={"alpha": 0.5, 's': 10})
plt.title('Origin Country Australia, SEM Non-Brand for Google')
plt.show()


In [ ]:

aust = result[result['dim_origin_region'] == 'Australia']
aust_google = result[(result['dim_origin_region'] == 'Australia') & (result['platform'] == 'google')]
aust_google_sem = result[(result['dim_origin_region'] == 'Australia') & (result['platform'] == 'google') & (result['channel'] == 'SEM Non-brand')]

def print_costs(df, df_name):
   
   # for the df, find the revenue/cost/clicks sum and print "df_name rev/cost/clicks: value" with 2 decimal places
   print ('{} revenue: {:.2f} '.format(df_name, df.revenue.sum()))
   print ('{} cost: {:.2f} '.format(df_name, df.cost.sum()))
   print ('{} clicks: {:.2f} '.format(df_name, df.clicks.sum()))
   print ('\n')
   
   # for result, find the revenue/cost/clicks sum and print "total rev/cost/clicks: value" with 2 decimal places
   print ('total revenue: {:.2f}'.format(result.revenue.sum()))
   print ('total cost: {:.2f}'.format(result.cost.sum()))
   print ('total clicks: {:.2f}'.format(result.clicks.sum()))
   print ('\n')
   
   # for the df/result, find the revenue/cost/clicks sum and print "df_name / total rev/cost/clicks: value" with 2 decimal places
   print ('{} / total revenume: {:.5f}'.format(df_name, df.revenue.sum() / result.revenue.sum()))
   print ('{} / total cost: {:.5f}'.format(df_name, df.cost.sum() / result.cost.sum()))
   print ('{} / total clicks: {:.5f}'.format(df_name, df.clicks.sum() / result.clicks.sum()))
   print ('--'*20)
   print ('\n')
   
print_costs(aust, 'australia')
print_costs(aust_google, 'australia_google')
print_costs(aust_google_sem, 'australia_google_sem')